## A. Configuraciones Generales.

In [ ]:
#1. Librerías.
%run "./librerias.ipynb"

In [ ]:
#2. Constantes.
#i. Generales.
%run "./constantes.ipynb"

#ii. Particulares.
#a. Mes de Test.
mes_test = mes_test
#b. Dataset post Future Engineering del modelo.
dataset_input = datasets_path + 'competencia_02_eugenio_negrin.parquet'
#c. Modelo optimizado del modelo.
modelo = modelos_path + 'lgbm_eugenio_negrin.txt'
#d. Path output con las predicciones del modelo para el período de interés.
dataset_output = path_output_m3_test

In [ ]:
#3. Funciones.
%run "./funciones.ipynb"

In [ ]:
#4. Lectura de dataframes post-feature engineering (.csv o .parquet), y modelos ya optimizados previamente (.txt).
data = pd.read_parquet(dataset_input)

In [ ]:
#5. Dividimos entre conjuntos de datos.
#i. Datos a predecirr.
test_data = data[data['foto_mes'] == mes_test]
#ii. Borramos las columnas de no interés.
X_test = test_data.drop(['clase_ternaria'], axis=1)

In [ ]:
#6. Obtenemos los hiperparámetros del modelo.
model_lgb = lgb.Booster(model_file=modelo)
params = model_lgb.params # Parámetros.
best_iter = params["num_iterations"]  # Mejor número de iteraciones guardado en el modelo.

## B. Predicción de Test.

In [ ]:
#i. Predecimos propiamente dicho.
predicciones = model_lgb.predict(X_test)

In [ ]:
#ii. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
X_test['probabilidad'] = predicciones

In [ ]:
#iii. Nos quedamos solamente con las columnas de interés.
stacking_predictions = X_test[['numero_de_cliente', 'foto_mes', 'probabilidad']]

In [ ]:
#iv. Renombramos.
stacking_predictions.rename({"probabilidad":"probabilidad_modelo3"},axis=1,inplace=True)

In [ ]:
#v. Exportamos.
stacking_predictions.to_parquet(dataset_output)